In [1]:
# 라이브러리
# 브라우저 제어를 위한 selenium
# selenium 본체
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
# selenium 키입력
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
# 키입력간 딜레이를 주기위한
import time
# 봇 탐지 우회를 위한 subprocess
import subprocess
# 크롬 브라우저 자동 버전 관리
from webdriver_manager.chrome import ChromeDriverManager
# 파일 디렉터리에 접근
import os
# 웹 드라이버 로드되도록 기다리기

# const
WAIT_INTERVAL = 3

##### path 설정 #####
browserPath = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
cookiePath = r"C:\Users\moonsung1\AppData\Local\Google\Chrome\User Data"
closetPath = r"C:\Users\moonsung1\Desktop\201826\public\uploads"
shoesDirName = "shoe"
topDirName = "top"
bottomDirName = "bottom"
outerDirName = "outer"
#####################
# 서브 프로세스로 크롬 브라우저 boot
subProc = subprocess.Popen(r'{0} --remote-debugging-port=9222 --user-data-dir="{1}"'.format(browserPath, cookiePath))
url = "https://chatgpt.com/"

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
driver.set_window_position(1529, 849)
time.sleep(WAIT_INTERVAL)


In [2]:
# chatGPT Class
class ChatGPT:
    
    def __init__(self, driver):
        self.GPT_URL = "https://chatgpt.com/"
        self.driver = driver
        self.newGPT()
        
    def newGPT(self):
        self.driver.get(self.GPT_URL)
        time.sleep(WAIT_INTERVAL * 2)
        self.getMainTag()

    def getMainTag(self):
        self.textBox = self.driver.find_element(By.CSS_SELECTOR, "#prompt-textarea")
        self.imgTag = self.driver.find_element(By.CSS_SELECTOR, "#__next > div.relative.z-0.flex.h-full.w-full.overflow-hidden > div.relative.flex.h-full.max-w-full.flex-1.flex-col.overflow-hidden > main > div.flex.h-full.flex-col.focus-visible\:outline-0 > div.w-full.md\:pt-0.dark\:border-white\/20.md\:border-transparent.md\:dark\:border-transparent.md\:w-\[calc\(100\%-\.5rem\)\].juice\:w-full > div.px-3.text-base.md\:px-4.m-auto.md\:px-5.lg\:px-1.xl\:px-5 > div > form > div > div.flex.w-full.items-center > div > div > div:nth-child(1) > div > input")
        self.sendButton = self.driver.find_element(By.CSS_SELECTOR, "#__next > div.relative.z-0.flex.h-full.w-full.overflow-hidden > div.relative.flex.h-full.max-w-full.flex-1.flex-col.overflow-hidden > main > div.flex.h-full.flex-col.focus-visible\:outline-0 > div.w-full.md\:pt-0.dark\:border-white\/20.md\:border-transparent.md\:dark\:border-transparent.md\:w-\[calc\(100\%-\.5rem\)\].juice\:w-full > div.px-3.text-base.md\:px-4.m-auto.md\:px-5.lg\:px-1.xl\:px-5 > div > form > div > div.flex.w-full.items-center > div > div > button")

    def canSend(self): # 주의할 점, 한 글자도 입력되지 않아도 보낼 수 없는 조건이다. 입력해놓고 대기할때 쓰자
        disabled = self.sendButton.get_attribute("disabled")
        stop_button = self.sendButton.get_attribute("data-testid")
        if(disabled or (stop_button == "fruitjuice-stop-button")): # send버튼이 disable이거나 data-testid가 중단이라면 False
            return False
        else: return True

    def isStopButton(self): # chatGPT가 결과물을 다 뱉었는지 보는 
        stop_button = self.sendButton.get_attribute("data-testid")
        if(stop_button == "fruitjuice-stop-button"):
            return True
        else: return False

    def waitWhileStopButton(self):
        time.sleep(WAIT_INTERVAL * 1)
        while(self.isStopButton()):
            time.sleep(WAIT_INTERVAL)
        time.sleep(WAIT_INTERVAL * 1) 
            
    def enter(self):
        ActionChains(self.driver).click(self.sendButton).perform()

    def inputText(self, text):
        self.textBox.send_keys(text)

    def inputImg(self, imgPath):
        self.imgTag.send_keys(imgPath)

    def inputImgs(self, imgPathList):
        imgPaths = "\n".join(imgPathList)
        self.inputImg(imgPaths)

    def fetchChats(self):
        self.chats = driver.find_elements(By.CSS_SELECTOR, "#__next > div.relative.z-0.flex.h-full.w-full.overflow-hidden > div.relative.flex.h-full.max-w-full.flex-1.flex-col.overflow-hidden > main > div.flex.h-full.flex-col.focus-visible\:outline-0 > div.flex-1.overflow-hidden > div > div > div > div > div")
                                                            
    def downloadResultImg(self):
        #button = self.chats[-1].find_element(By.CSS_SELECTOR, "div > div > div.group\/conversation-turn.relative.flex.w-full.min-w-0.flex-col.agent-turn > div > div.flex.flex-grow.flex-col.max-w-full > div.grid.gap-2.grid-cols-1.my-1.transition-opacity.duration-300 > div > div > div > div.relative.h-full > div.invisible.absolute.top-3.flex.gap-1.group-hover\/dalle-image\:visible.right-3 > button")
        button = self.chats[-1].find_element(By.CSS_SELECTOR, "div div.invisible.absolute.top-3.flex.gap-1.group-hover\/dalle-image\:visible.right-3 > button")
        ActionChains(self.driver).click(button).perform()
        

    def downloadResultText(self):
        #result = self.chats[-1].find_element(By.CSS_SELECTOR, "div > div > div.group\/conversation-turn.relative.flex.w-full.min-w-0.flex-col.agent-turn > div > div.flex.flex-grow.flex-col.max-w-full > div > div > div > p")
        result = self.chats[-1].find_element(By.CSS_SELECTOR, "div div.flex.flex-grow.flex-col.max-w-full > div > div > div > p")
        resultText = result.get_attribute("innerText")
        textList = resultText.replace("[", ",").replace("]", ",").split(",")
        # 안 쓰는 앞 뒤 제거
        textList.pop(-1)
        textList.pop(0)
        resultList = list(map(int, textList))
        return resultList

#__next > div.relative.z-0.flex.h-full.w-full.overflow-hidden > div > main > div.flex.h-full.flex-col.focus-visible\:outline-0 > div.flex-1.overflow-hidden > div > div > div > div > div:nth-child(3) > div > div > div.group\/conversation-turn.relative.flex.w-full.min-w-0.flex-col.agent-turn > div > div.flex.flex-grow.flex-col.max-w-full > div > div > div > p

In [3]:
class Closet:
    def __init__(self, closetPath):
        self.closetPath = closetPath
        ##### 옷장 아래에 신발(shoes), 탑(top), 바텀(bottom) 폴더명 변경 해야합니다 #####
        self.shoesPath =r"{0}\{1}".format(closetPath, shoesDirName)
        self.topPath =r"{0}\{1}".format(closetPath, topDirName)
        self.bottomPath =r"{0}\{1}".format(closetPath, bottomDirName)
        self.outerPath =r"{0}\{1}".format(closetPath, outerDirName)

        self.fetchShoesList()
        self.fetchTopList()
        self.fetchBottomList()
        self.fetchOuterList()
        
    def fetchShoesList(self):
        self.shoesList = os.listdir(self.shoesPath)
        self.shoesList_noXtender = list(map(self.deleteJPGxtender, self.shoesList))
        
    def fetchTopList(self):
        self.topList = os.listdir(self.topPath)
        self.topList_noXtender = list(map(self.deleteJPGxtender, self.topList))
        
    def fetchBottomList(self):
        self.bottomList = os.listdir(self.bottomPath)
        self.bottomList_noXtender = list(map(self.deleteJPGxtender, self.bottomList))

    def fetchOuterList(self):
        self.outerList = os.listdir(self.outerPath)
        self.outerList_noXtender = list(map(self.deleteJPGxtender, self.outerList))
        
    def deleteJPGxtender(self, fileName):
        tmp_file = fileName.split(".")
        return tmp_file[0]

    def shoesListToString(self):
        indexedShoesList = []
        for index, shoes in enumerate(self.shoesList_noXtender):
            indexedShoes = f"{index}. {shoes}"
            indexedShoesList.append(indexedShoes)

        return ", ".join(indexedShoesList)
        
    def topListToString(self):
        indexedTopList = []
        for index, top in enumerate(self.topList_noXtender):
            indexedTop = f"{index}. {top}"
            indexedTopList.append(indexedTop)

        return ", ".join(indexedTopList)
        
    def bottomListToString(self):
        indexedBottomList = []
        for index, bottom in enumerate(self.bottomList_noXtender):
            indexedBottom = f"{index}. {bottom}"
            indexedBottomList.append(indexedBottom)

        return ", ".join(indexedBottomList)

    def outerListToString(self):
        indexedOuterList = []
        for index, outer in enumerate(self.outerList_noXtender):
            indexedOuter = f"{index}. {outer}"
            indexedOuterList.append(indexedOuter)

        return ", ".join(indexedOuterList)
        

In [4]:
# chatGPT 생성
chatGPT = ChatGPT(driver)
time.sleep(WAIT_INTERVAL)

In [5]:
# 임의 옷장 하나 생성
closet = Closet(closetPath)

In [6]:
requestText = ""
requestText += "내가 가지고 있는 옷을 알려주면, 오늘의 날씨, 기온,  색깔 조합(옷 조합의 디자인) 세 가지를 고려해서 옷 조합을 추천해줘 "
requestText += "내가 가진 옷은 다음과 같아. "
requestText += "(신발: " + closet.shoesListToString()
requestText += ") (상의: " + closet.topListToString()
requestText += ") (하의: " + closet.bottomListToString()
requestText += ") (아우터: " + closet.outerListToString()
requestText += ") 추천하는 옷 조합을 신발, 상의, 하의, 아우터 순서대로 번호를 GPTrecommended 리스트에 담아줘. 예를 들면 GPTrecommended = [0, 2, 1, 3] 이렇게. "
requestText += "이유가 있다면 아우터는 선택하지 않아도 돼 특히 날씨가 덥거나 하다면. 선택하지 않으면 -1을 리스트에 담아줘. "
requestText += "대답은 GPTrecommended만 해 아주 짧게.GPTrecommended 리스트만 알려주고 다른 건 말 하지마."

In [7]:
# 요청 보내기
chatGPT.inputText(requestText)
chatGPT.enter()

chatGPT.waitWhileStopButton()

In [8]:
# 결과물 가져오기 
chatGPT.fetchChats()
chosenClothes = chatGPT.downloadResultText()
time.sleep(WAIT_INTERVAL)

chosenShoesIndex = chosenClothes[0]
chosenTopIndex = chosenClothes[1]
chosenBottomIndex = chosenClothes[2]
chosenOuterIndex = chosenClothes[3]

In [9]:
# 고른 걸로 옷 그려줘(옷 업로드)
shoesPath = r"{0}\{1}".format(closet.shoesPath, closet.shoesList[chosenShoesIndex])
topPath = r"{0}\{1}".format(closet.topPath, closet.topList[chosenTopIndex])
bottomPath = r"{0}\{1}".format(closet.bottomPath, closet.bottomList[chosenBottomIndex])
pathList = [shoesPath, topPath, bottomPath]
if(chosenOuterIndex != -1):
    outerPath = r"{0}\{1}".format(closet.outerPath, closet.outerList[chosenOuterIndex])
    pathList.append(outerPath)

chatGPT.inputImgs(pathList)

chatGPT.waitWhileStopButton()

In [10]:
# 고른 걸로 옷 그려줘(텍스트로 업로드)
chosenShoesName = closet.shoesList_noXtender[chosenShoesIndex]
chosenTopName = closet.topList_noXtender[chosenTopIndex]
chosenBottomName = closet.bottomList_noXtender[chosenBottomIndex]

chosenNameList = [chosenShoesName, chosenTopName, chosenBottomName]
if(chosenOuterIndex != -1):
    chosenOuterName = closet.outerList_noXtender[chosenOuterIndex]
    chosenNameList.append(chosenOuterName)

chosenNameString = ", ".join(chosenNameList)

chatGPT.inputText(f"{chosenNameString}을(를) 입은 잘생긴 남자 한 명을 전신이 잘 보이게 그려줘. 그리고 날씨와 디자인에 관해서 선정한 이유를 알려줘. 날씨와 디자인으로 문단처럼 나누지 말고 하나의 단락으로 적어줘. 한국어로 답해")
time.sleep(WAIT_INTERVAL)
chatGPT.enter()

chatGPT.waitWhileStopButton()
time.sleep(WAIT_INTERVAL * 1)

In [11]:
# 옷 다운로드 받기
chatGPT.fetchChats()
chatGPT.downloadResultImg()
time.sleep(WAIT_INTERVAL * 1)

In [12]:
chatGPT_answer = chatGPT.chats[-1].get_attribute("innerText")


In [13]:
chatGPT_answer_br = chatGPT_answer.replace("\n", "")
chatGPT_answer_br = chatGPT_answer_br.replace("4o", "")
chatGPT_answer_br = chatGPT_answer_br.replace("ChatGPT", "")

In [14]:
chatGPT_answer_br

'오늘 서울의 날씨는 흐리고 기온은 약 23도 정도로, 비교적 시원하고 습도가 높은 편입니다. 이러한 날씨에는 흰색 스니커즈, 검정 티셔츠, 그리고 회색 트레이닝 바지를 추천합니다. 흰색 스니커즈는 다양한 스타일과 잘 어울리며, 검정 티셔츠는 기본적이면서도 깔끔한 인상을 줍니다. 회색 트레이닝 바지는 편안하면서도 캐주얼한 느낌을 주어 전체적으로 자연스럽고 세련된 코디를 완성할 수 있습니다. 이러한 조합은 흐린 날씨에 어울리면서도 활동하기 편리한 옷차림입니다.'

In [15]:
with open(r"C:\Users\moonsung1\Desktop\201826\public\comment.txt", "w", encoding="utf-8") as file:
    file.write(chatGPT_answer_br)


In [16]:
subProc.terminate()
time.sleep(1)